In [16]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
import Utilities

import torch
import torchvision

In [17]:
import Utilities
Utilities.downloadPEALViT()
Utilities.unzipSquareImages()

In [19]:
import torch
import torch.nn as nn
from torchvision.models.vision_transformer import VisionTransformer

ckpt_path   = "vit.cpl"
device      = "cuda" if torch.cuda.is_available() else "cpu"

state_raw = torch.load(ckpt_path, map_location="cpu")
state = {k.replace("model.", "", 1): v for k, v in state_raw.items()}

model = VisionTransformer(
    image_size = 64,
    patch_size = 16,
    num_layers = 12,
    num_heads  = 12,
    hidden_dim = 768,
    mlp_dim    = 3072,
    dropout    = 0.0,
    num_classes = 0,
)

model.heads = nn.Identity()

model.load_state_dict(state, strict=False)
model = model.to(device).eval()

In [34]:
import pandas as pd, os, torch, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (precision_recall_fscore_support, accuracy_score, average_precision_score)
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

csv_path = "../Images/Square/data.csv"
df = pd.read_csv(csv_path)
label_cols = ["ClassA","ClassB","ClassC","ClassD"]

label_map = {
    name: row[label_cols].values.astype("float32")
    for name, row in df.set_index("Name").iterrows()
}

loader = Utilities.createImageDataloader(
    path="/Users/mawy/Desktop/Square Images",
    model_name="vit_b_16",
    image_size=64,
    batch_size=64,
    shuffle=False, 
    drop_last=False,
)

dataset = loader.dataset      

device = next(model.parameters()).device
model.eval()

X_list, y_list = [], []
idx_offset = 0
model.eval()

with torch.inference_mode():
    for xb in loader:
        feats = model(xb.to(device))
        X_list.append(feats.cpu())

        paths = [os.path.basename(loader.dataset.image_paths[i])
                 for i in range(idx_offset, idx_offset + xb.size(0))]
        idx_offset += xb.size(0)

        labels_batch = [label_map[p] for p in paths]
        y_list.append(torch.tensor(labels_batch))

X = torch.cat(X_list).numpy()       
Y = torch.cat(y_list).numpy()        

X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.2, random_state=42)

clf = OneVsRestClassifier(
        LogisticRegression(max_iter=1000, solver="lbfgs", n_jobs=-1)
      ).fit(X_tr, y_tr)

probs_te = clf.predict_proba(X_te)
probs_tr = clf.predict_proba(X_tr)

aps_te  = [average_precision_score(y_te[:, i], probs_te[:, i]) for i in range(4)]
pred_bin_te = (probs_te >= 0.5).astype(int)
acc_cls_te  = (pred_bin_te == y_te).mean(axis=0)
acc_mean_te = acc_cls_te.mean()
prec_te, rec_te, _, _ = precision_recall_fscore_support(
    y_te, pred_bin_te, average=None, zero_division=0
)

aps_tr  = [average_precision_score(y_tr[:, i], probs_tr[:, i]) for i in range(4)]
pred_bin_tr = (probs_tr >= 0.5).astype(int)
acc_cls_tr  = (pred_bin_tr == y_tr).mean(axis=0)
acc_mean_tr = acc_cls_tr.mean()
prec_tr, rec_tr, _, _ = precision_recall_fscore_support(
    y_tr, pred_bin_tr, average=None, zero_division=0
)

print("Average Accuracy pro Klasse")
for col, a_tr, a_te in zip(label_cols, acc_cls_tr, acc_cls_te):
    print(f"  {col:7s}: {a_tr:0.3f} (Train), {a_te:0.3f} (Test)")
print(f"Mean Accuracy = {acc_mean_tr:0.3f} (Train), {acc_mean_te:0.3f} (Test)\n")

print("Average Precision (AP) pro Klasse")
for col, ap_tr, ap_te in zip(label_cols, aps_tr, aps_te):
    print(f"  {col:7s}: {ap_tr:0.3f} (Train), {ap_te:0.3f} (Test)")
print(f"Mean AP       = {np.mean(aps_tr):0.3f} (Train), {np.mean(aps_te):0.3f} (Test)\n")

print("Average Recall pro Klasse")
for col, r_tr, r_te in zip(label_cols, rec_tr, rec_te):
    print(f"  {col:7s}: {r_tr:0.3f} (Train), {r_te:0.3f} (Test)")
print(f"Mean Recall   = {rec_tr.mean():0.3f} (Train), {rec_te.mean():0.3f} (Test)")

Average Accuracy pro Klasse
  ClassA : 0.957 (Train), 0.946 (Test)
  ClassB : 1.000 (Train), 0.998 (Test)
  ClassC : 0.690 (Train), 0.641 (Test)
  ClassD : 0.690 (Train), 0.635 (Test)
Mean Accuracy = 0.834 (Train), 0.805 (Test)

Average Precision (AP) pro Klasse
  ClassA : 0.994 (Train), 0.989 (Test)
  ClassB : 1.000 (Train), 1.000 (Test)
  ClassC : 0.762 (Train), 0.702 (Test)
  ClassD : 0.770 (Train), 0.701 (Test)
Mean AP       = 0.881 (Train), 0.848 (Test)

Average Recall pro Klasse
  ClassA : 0.955 (Train), 0.941 (Test)
  ClassB : 1.000 (Train), 0.999 (Test)
  ClassC : 0.691 (Train), 0.625 (Test)
  ClassD : 0.687 (Train), 0.616 (Test)
Mean Recall   = 0.833 (Train), 0.795 (Test)


In [ ]:
torch.load()